# LSTM Model


## Import Data
### Players from RPS_game.py

In [ ]:
import random


def play(player1, player2, num_games, verbose=False):
    p1_prev_play = ""
    p2_prev_play = ""
    results = {"p1": 0, "p2": 0, "tie": 0}

    for _ in range(num_games):
        p1_play = player1(p2_prev_play)
        p2_play = player2(p1_prev_play)

        if p1_play == p2_play:
            results["tie"] += 1
            winner = "Tie."
        elif (p1_play == "P" and p2_play == "R") or (
                p1_play == "R" and p2_play == "S") or (p1_play == "S"
                                                       and p2_play == "P"):
            results["p1"] += 1
            winner = "Player 1 wins."
        elif p2_play == "P" and p1_play == "R" or p2_play == "R" and p1_play == "S" or p2_play == "S" and p1_play == "P":
            results["p2"] += 1
            winner = "Player 2 wins."

        if verbose:
            print("Player 1:", p1_play, "| Player 2:", p2_play)
            print(winner)
            print()

        p1_prev_play = p1_play
        p2_prev_play = p2_play

    games_won = results['p2'] + results['p1']

    if games_won == 0:
        win_rate = 0
    else:
        win_rate = results['p1'] / games_won * 100

    print("Final results:", results)
    print(f"Player 1 win rate: {win_rate}%")

    return (win_rate)


def quincy(prev_play, counter=[0]):

    counter[0] += 1
    choices = ["R", "R", "P", "P", "S"]
    return choices[counter[0] % len(choices)]


def mrugesh(prev_opponent_play, opponent_history=[]):
    opponent_history.append(prev_opponent_play)
    last_ten = opponent_history[-10:]
    most_frequent = max(set(last_ten), key=last_ten.count)

    if most_frequent == '':
        most_frequent = "S"

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[most_frequent]


def kris(prev_opponent_play):
    if prev_opponent_play == '':
        prev_opponent_play = "R"
    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prev_opponent_play]


def abbey(prev_opponent_play,
          opponent_history=[],
          play_order=[{
              "RR": 0,
              "RP": 0,
              "RS": 0,
              "PR": 0,
              "PP": 0,
              "PS": 0,
              "SR": 0,
              "SP": 0,
              "SS": 0,
          }]):

    if not prev_opponent_play:
        prev_opponent_play = 'R'
    opponent_history.append(prev_opponent_play)

    last_two = "".join(opponent_history[-2:])
    if len(last_two) == 2:
        play_order[0][last_two] += 1

    potential_plays = [
        prev_opponent_play + "R",
        prev_opponent_play + "P",
        prev_opponent_play + "S",
    ]

    sub_order = {
        k: play_order[0][k]
        for k in potential_plays if k in play_order[0]
    }

    prediction = max(sub_order, key=sub_order.get)[-1:]

    ideal_response = {'P': 'S', 'R': 'P', 'S': 'R'}
    return ideal_response[prediction]


def human(prev_opponent_play):
    play = ""
    while play not in ['R', 'P', 'S']:
        play = input("[R]ock, [P]aper, [S]cissors? ")
        print(play)
    return play


def random_player(prev_opponent_play):
    return random.choice(['R', 'P', 'S'])


## Generate data from Players
generate 10 games of 1000 from each player with winning response output

In [ ]:
def GenerateGames(player, opponent, game_count = 10, game_length = 1000, cheater = False):
    if(cheater):
        player = TelepathicPlayer
    
    filename = "Games/" + player.__name__ + "_vs_" + opponent.__name__ + "_game_"
    for i in range(0 , game_count):
        f = open(filename + str(i) + ".txt", "w")
        p1_prev_play = ""
        p2_prev_play = ""
        for _ in range(game_length):
            
            p2_play = opponent(p1_prev_play)
            if(cheater):
                p1_play = player(p2_play)
            else:
                p1_play = player(p2_prev_play)
            player_win = 0
            if (p1_play == "P" and p2_play == "R") or \
                (p1_play == "R" and p2_play == "S") or \
                (p1_play == "S" and p2_play == "P"):
                player_win = 1
            f.write(p1_play + "," + p2_play + "," + str(player_win) + '\n')
            
            p1_prev_play = p1_play
            p2_prev_play = p2_play
        f.close()
        
def TelepathicPlayer(opponent_guess):
    if(opponent_guess == 'R'):
        return 'P'
    elif(opponent_guess == 'P'):
        return 'S'
    elif(opponent_guess == 'S'):
        return 'R'


In [ ]:
#GenerateGames(random_player, quincy)
#GenerateGames(random_player, abbey)
#GenerateGames(random_player, kris)
#GenerateGames(random_player, mrugesh)
GenerateGames(TelepathicPlayer, quincy, cheater = True)
GenerateGames(TelepathicPlayer, abbey, cheater = True, game_count = 10)
GenerateGames(TelepathicPlayer, kris, cheater = True)
GenerateGames(TelepathicPlayer, mrugesh, cheater = True)

## Read and Concatenate Games
Take all games and load into dataframe

In [ ]:
import pandas as pd
from os import listdir
from os.path import isfile, join

path = "Games/"
all_game_paths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
games_concat = pd.DataFrame(columns = ["Player","Opponent","Result"])
games = []
for game_path in all_game_paths:
    #print(game_path)
    game_data = pd.read_csv(game_path, names = ["Player","Opponent","Result"])
    games.append(game_data)
    games_concat = games_concat.append(game_data)
    
#print(len(game_data.index)) 
#game_data.head()  
print(len(games_concat.index))
games_concat.head()


# LSTM Training
### Parameters

In [ ]:
seq_length = 100 #how far back the ltsm looks when training
#Model parameters
BATCH_SIZE = 64
VOCAB_SIZE = 3 # RPS
EMBEDDING_DIM = 128
RNN_UNITS = 512

TRAINING_EPOCHS = 100

#best so far
#100
#64
#3
#128
#512
#200


### Format data for LTSM training

In [ ]:
import tensorflow as tf

def encode(df):
    df = df.replace('R', 0)
    df = df.replace('P', 1)
    df = df.replace('S', 2)
    return df

def encodeRPS(char):
    if char == 'R':
        return 0
    elif char == 'P':
        return 1
    elif char == 'S':
        return 2
    
def decodeRPS(num):
    if num == 0:
        return 'R'
    elif num == 1:
        return 'P'
    elif num == 2:
        return 'S'
    
temp = games_concat.drop("Result", axis = 1)
temp = encode(temp)
input_data = tf.data.Dataset.from_tensor_slices(temp["Opponent"])
target_data = tf.data.Dataset.from_tensor_slices(temp["Player"])
print(temp)
print(input_data)

#split into sequences
input_sequences = input_data.batch(seq_length+1, drop_remainder = True)
target_sequences = target_data.batch(seq_length+1, drop_remainder = True)
print(input_sequences)

#remove last element from all input sequences
def remove_last(chunk):
    return chunk[:-1]
inputs = input_sequences.map(remove_last)

#Code for abbey training
#inputs = target_sequences.map(remove_last)

#remove first element from all target sequences
def remove_first(chunk):
    return chunk[1:]
targets = target_sequences.map(remove_first)

dataset = tf.data.Dataset.zip((inputs, targets))
for element in dataset:
    print(element)
    break


### Build the model

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, LSTM, Embedding

In [ ]:
#Shuffle sequences before batching
data = dataset.shuffle(10000).batch(BATCH_SIZE, drop_remainder = True)

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.Sequential()
    model.add(Embedding(vocab_size, embedding_dim, batch_input_shape = [batch_size, None]))
    model.add(LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size+1))
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

### Create loss function

In [ ]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)

### Compile the model

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Create Checkpoints
configure the model to create checkpoints as it trains so the model can be reloaded and continue training

In [ ]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath = checkpoint_prefix, save_weights_only = True)

print(tf.__version__)

### Training the model

In [ ]:
import os
#Program can crash when training due to tensorflow 2.0 GPU issue

history = model.fit(data, epochs = TRAINING_EPOCHS, callbacks = [checkpoint_callback])

### Rebuild model for predicting one output rather than a vector

In [ ]:
#change config
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size = 1)
#load trained weights from checkpoint
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

## Generate Outputs from Model

In [ ]:
def Get_Prediction(model, opponent_history):
    #format history for input
    input_eval = [0 if c == '' else encodeRPS(c) for c in opponent_history]
    input_eval = tf.expand_dims(input_eval, 0)
    
    #high temperature  => suprising output
    #low temperature => predictable output
    temperature = 1; 
    
    model.reset_states()
    
    prediction = model(input_eval)
    prediction = tf.squeeze(prediction, 0)
    
    prediction = prediction / temperature
    predicted_id = tf.random.categorical(prediction, num_samples = 1)[-1,0].numpy()
    
    return decodeRPS(predicted_id)

#print(Get_Prediction(model, ['R','R','P']))

# RPS.py
Player function

In [ ]:
plays = 1000

def player(prev_play, opponent_history=[]):
    if len(opponent_history) >= plays: 
        opponent_history.clear()
        
    opponent_history.append(prev_play)

    guess = Get_Prediction(model, opponent_history)
    
    return guess

# MAIN.PY

In [ ]:
play(player, quincy, plays)
play(player, abbey, plays)
play(player, kris, plays)
play(player, mrugesh, plays)

## Saving the model

In [ ]:
model.save('Models/General')